In [1]:
import matplotlib.pyplot as plt
import numpy as np
import matplotlib
import random
from operator import itemgetter

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier
from sklearn.svm import LinearSVC

import cv2

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (10.0, 8.0)

from itertools import count
iid = count()

ModuleNotFoundError: No module named 'cv2'

### 0. Preprocessing

In [ ]:
mini_X = np.loadtxt("./Datasets/mini_train_x.csv", delimiter=",")
mini_y = np.loadtxt("./Datasets/mini_train_y.csv", delimiter=",")

# mini_X = np.loadtxt("./Datasets/train_x.csv", delimiter=",")
# mini_y = np.loadtxt("./Datasets/train_y.csv", delimiter=",")

#### Reshape to easily visualize while cleaning data

In [ ]:
%%time
x = mini_X.reshape(-1, 64, 64)

imgs = []
for idx in range(len(x)):
    img_id = next(iid)
    cv2.imwrite('Output/{}.jpg'.format(img_id), 255-x[idx])
    imgs.append(cv2.imread('Output/{}.jpg'.format(img_id), 0))

We will first transform to binary image (black, white):

In [ ]:
%%time
bin_imgs = []
for img in imgs:
    _, bin_img = cv2.threshold(img, 50, 255, cv2.THRESH_BINARY)
    bin_imgs.append(bin_img)

Then we will remove the noice by dilatating and eroding the image:

In [ ]:
denoised_imgs = []
for bin_img in bin_imgs:
    dilatated = cv2.dilate(bin_img, np.ones((3,3)))
    denoised_imgs.append(cv2.erode(dilatated, np.ones((2,2))))

Lets visualize the transformations:

In [ ]:
def plot_images(images, titles):
    cols = len(images)
    rows = len(images[0])
    for r in range(rows):
        for c in range(cols):
            plt.subplot(1,cols,c+1)
            plt.imshow(images[c][r],'gray')
            plt.title(titles[c])
            plt.xticks([]),plt.yticks([])
        
        plt.show()
        
titles = ['Original Image', 'Binary', 'Dilatated & Eroded']
images = [imgs[:5], bin_imgs[:5], denoised_imgs[:5]]
plot_images(images, titles)

Finally we will blur in order to smooth any edges __(TEST IF THIS IS ACTUALLY USEFUL)__

In [ ]:
blurred_imgs = []
for denoised_img in denoised_imgs:
    blurred_imgs.append(cv2.blur(denoised_img,(2,2)))

random_idx = 4
blur = cv2.blur(denoised_imgs[random_idx],(2,2))
gaussian_blur = cv2.GaussianBlur(denoised_imgs[random_idx],(3,3),0)    
plt.subplot(151),plt.imshow(imgs[random_idx],'gray'),plt.title('Original')
plt.xticks([]), plt.yticks([])
plt.subplot(152),plt.imshow(bin_imgs[random_idx],'gray'),plt.title('Binary')
plt.xticks([]), plt.yticks([])
plt.subplot(153),plt.imshow(denoised_imgs[random_idx],'gray'),plt.title('Denoised')
plt.xticks([]), plt.yticks([])
plt.subplot(154),plt.imshow(gaussian_blur,'gray'),plt.title('Gaussian Blur')
plt.xticks([]), plt.yticks([])
plt.subplot(155),plt.imshow(blur,'gray'),plt.title('Pixel Blur')
plt.xticks([]), plt.yticks([])
plt.show()

### 1. Linear Learners

In [ ]:
# hyperparameter permutations
SEED = 42    # random state seed for consistent results
FOLDS = 3    # number of folds for K-fold cross validation
DUAL = [False, True] # WHEN n_samples > n_features DUAL=FALSE
LOSSES = ['squared_hinge','hinge']
PENALTIES = ['l1','l2']
SOLVERS = ['lbfgs'] # 'newton-cg','sag','saga',
# error_coefs = np.logspace(-0.01, 2, 15)
# FOUND OUT A SMALLER RANGE OF GOOD VALUES
error_coefs = np.linspace(20,80, 10)

KFOLD = KFold(n_splits=FOLDS, random_state=SEED)

clean_X = [img.flatten() for img in denoised_imgs]
mini_X_train, mini_X_test, mini_y_train, mini_y_test = train_test_split(clean_X, mini_y, 
                                                                        test_size=0.3,
                                                                        random_state=SEED)

#### Helper methods
Methods to generate all possible permutations of HyperParameters 
for LinearSVMs and Logistic Regressor:

In [ ]:
def get_svm_hp_permutations():
    clfs = []
    for d in DUAL:
        for l in LOSSES:
            for p in PENALTIES:
                # ignore unvalid combinations
                if not d and p == 'l2': continue
                if p == 'l1' and l == 'hinge': continue
                if d and p == 'l1' and l == 'squared_hinge': continue

                for c in error_coefs:
                    clfs.append(LinearSVC(C=c,dual=d,loss=l,penalty=p))
    
    return clfs

def get_logreg_hp_permutations():
    clfs = []
#     for d in DUAL:
    for p in PENALTIES:
        for s in SOLVERS:
            # ignore unvalid combinations
#             if not d and p == 'l2': continue
            if s != 'saga' and p == 'l1': continue

            for c in error_coefs:
                clfs.append(LogisticRegression(C=c,penalty=p,solver=s,n_jobs=8))
    
    return clfs

#### Base and dummy clf performances for reference

In [ ]:
# dummy clf performance
random_clf = DummyClassifier(random_state=42)
result = cross_val_score(random_clf, mini_X_train, mini_y_train, cv=KFOLD, scoring='f1_micro')
print('Random clf performance: {:.4f}'.format(result.mean()))

# Base performance (default HP)
result = cross_val_score(LinearSVC(), mini_X_train, mini_y_train, cv=KFOLD, scoring='f1_micro')
print('LinearSVM base performance: {:.4f}'.format(result.mean()))

result = cross_val_score(LogisticRegression(), mini_X_train, mini_y_train, cv=KFOLD, scoring='f1_micro')
print('LogReg base performance: {:.4f}'.format(result.mean()))

#### Perform Model Selection for LinearSM

In [ ]:
scores = []
# create preprocessing pipeline
for specific_svm_permutation in get_svm_hp_permutations():
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('clf', specific_svm_permutation))
    model = Pipeline(estimators)
    result = cross_val_score(model, mini_X_train, mini_y_train, cv=KFOLD, scoring='f1_micro')

    scores.append((specific_svm_permutation, result.mean()))
    
best_svm, max_train = max(scores,key=itemgetter(1)) 
print('Best SVM: \n{}'.format(best_svm))
print('TRAIN SCORE: {:.4f}'.format(max_train))

#### Perform Model Selection for LogisticRegression

In [ ]:
scores = []
# create preprocessing pipeline
for specific_logreg_permutation in get_logreg_hp_permutations():
    estimators = []
    estimators.append(('standardize', StandardScaler()))
    estimators.append(('clf', specific_logreg_permutation))
    model = Pipeline(estimators)
    result = cross_val_score(model, mini_X_train, mini_y_train, cv=KFOLD, scoring='f1_micro')

    scores.append((specific_logreg_permutation, result.mean()))
    
best_svm, max_train = max(scores,key=itemgetter(1)) 

print('Best SVM: \n{}'.format(best_svm))
print('TRAIN SCORE: {:.4f}'.format(max_train))